In [7]:
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import pandas as pd

# pycaret 이용

## 모든 종목 예측

* 종가와 상관계수 높은 특성 : KOSDAQ(0.87), KOSPI(0.79), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
* 예측 순서(종가와 상관계수 높은 특성 먼저 예측)
    * weekday, weeknumdm로 KOSDAQ 예측(종가와 상관계수 0.87)
    * weekday, weeknum, KOSDAQ으로 NASDAQ 예측(KOSDAQ과 상관계수 0.68)
    * weekday, weeknum, KOSDAQ, NASDAQ으로 S&P500 예측(NASDAQ과 상관계수 0.93)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500으로 DOW 예측(S&P500과 상관계수 0.95)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW로 KOSPI 예측
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW, KOSPI로 다른 종가 예측 

In [8]:
from catboost import CatBoostRegressor

In [9]:
cat = CatBoostRegressor()

In [35]:
# 외부 데이터 가져오기
df = pd.read_csv('../data/20210104_20211119_data_interpolate.csv')
df.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           230 non-null    object 
 1   exchange_rate  230 non-null    float64
 2   kospi          230 non-null    float64
 3   kosdaq         230 non-null    float64
 4   kospi100       230 non-null    float64
 5   dow            230 non-null    float64
 6   nasdaq         230 non-null    float64
 7   sp500          230 non-null    float64
 8   미국채10년-2년      230 non-null    float64
 9   BTC            230 non-null    float64
 10  Gold           230 non-null    float64
 11  Oil            230 non-null    float64
 12  US10Y          230 non-null    float64
 13  US2Y           230 non-null    float64
dtypes: float64(13), object(1)
memory usage: 25.3+ KB


In [37]:
# 종목 정보 가져오기
path = '../data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [38]:
# 종목 코드 6자리로 맞추기
stock_list['종목코드'] = stock_list["종목코드"].astype(str).str.zfill(6)
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI


# 종가 예측

* 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 종가

In [39]:
start_date = '20210104'
end_date = '20211119'
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

# 종가 예측

#### 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 다른 종가

In [40]:
# 종목 주가 정보 가져오기
data = fdr.DataReader('005930', start = start_date, end = end_date)[['Close']].reset_index()
# 공휴일 포함한 평일 날짜로 데이터 셋 채우기
data = pd.merge(Business_days, data, how = 'outer')
# 요 번호와 주 번호 지정하여 열로 추가하기
data['weekday'] = data.Date.apply(lambda x : x.weekday())
data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
# 빈값 선형으로 채우기
data.Close = data.Close.interpolate()
# 날짜 기준으로 외부 데이터와 주가 정보 합치기
data['Date'] = data['Date'].astype('str')
data = pd.merge(data, df, on='Date')
data.head()

,Date,Close,weekday,weeknum,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,83000.0,0,01,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,83900.0,1,01,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,82200.0,2,01,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,82900.0,3,01,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,88800.0,4,01,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [41]:
# 훈련, 테스트 셋 나누기
# 상관 관계가 높은 5개 특성만 이용 : KOSPI(0.79), KOSDAQ(0.87), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
mod_data = data[['Date','Close','weekday','weeknum','kospi','kosdaq','dow','nasdaq','sp500']]
mod_data['weeknum'] = mod_data['weeknum'].astype(int)
# 훈련 셋 : 2021년 1월 4일 ~ 10월 29일
train_data = mod_data.loc[mod_data['weeknum'].astype(int) < 44]
X_train = train_data[['weekday','weeknum']]
y_kosdaq = train_data['kosdaq']
y_train = train_data['Close']
# 테스트 셋 : 2021년 11월 1일 ~ 5일
test_data = mod_data.loc[mod_data['weeknum'].astype(int) == 44]
X_test = test_data[['weekday','weeknum']]
y_test = test_data['Close']

<ipython-input-41-297aae0c695d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_data['weeknum'] = mod_data['weeknum'].astype(int)


In [42]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((215, 2), (215,), (5, 2), (5,))

In [43]:
mod_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 229
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     230 non-null    object 
 1   Close    230 non-null    float64
 2   weekday  230 non-null    int64  
 3   weeknum  230 non-null    int32  
 4   kospi    230 non-null    float64
 5   kosdaq   230 non-null    float64
 6   dow      230 non-null    float64
 7   nasdaq   230 non-null    float64
 8   sp500    230 non-null    float64
dtypes: float64(6), int32(1), int64(1), object(1)
memory usage: 17.1+ KB


In [44]:
# catboost 훈련
# kosdaq 예측
cat.fit(X_train, y_kosdaq)
kosdaq_pred = cat.predict(X_train)
X_train['kosdaq'] = kosdaq_pred
test_kosdaq_pred = cat.predict(X_test)
X_test['kosdaq'] = test_kosdaq_pred

Learning rate set to 0.032115
0:	learn: 38.4306948	total: 4.08ms	remaining: 4.08s
1:	learn: 37.6950469	total: 6.31ms	remaining: 3.15s
2:	learn: 37.0497617	total: 8.04ms	remaining: 2.67s
3:	learn: 36.3590278	total: 9.73ms	remaining: 2.42s
4:	learn: 35.7909963	total: 11.5ms	remaining: 2.28s
5:	learn: 35.1481058	total: 13ms	remaining: 2.15s
6:	learn: 34.4965401	total: 14.6ms	remaining: 2.07s
7:	learn: 33.8862503	total: 16.2ms	remaining: 2s
8:	learn: 33.3780580	total: 17.8ms	remaining: 1.96s
9:	learn: 32.8360925	total: 19.4ms	remaining: 1.92s
10:	learn: 32.4713686	total: 20.9ms	remaining: 1.87s
11:	learn: 31.9921590	total: 22.5ms	remaining: 1.85s
12:	learn: 31.4803596	total: 24.3ms	remaining: 1.84s
13:	learn: 31.0381994	total: 26.3ms	remaining: 1.85s
14:	learn: 30.5430535	total: 28ms	remaining: 1.84s
15:	learn: 30.1385336	total: 28.9ms	remaining: 1.77s
16:	learn: 29.6869450	total: 29.5ms	remaining: 1.71s
17:	learn: 29.2570420	total: 30.2ms	remaining: 1.64s
18:	learn: 28.9661923	total: 30.8

207:	learn: 11.7823393	total: 136ms	remaining: 516ms
208:	learn: 11.7649496	total: 136ms	remaining: 515ms
209:	learn: 11.7233056	total: 137ms	remaining: 514ms
210:	learn: 11.6901142	total: 137ms	remaining: 513ms
211:	learn: 11.6542589	total: 138ms	remaining: 513ms
212:	learn: 11.6158441	total: 138ms	remaining: 511ms
213:	learn: 11.5755718	total: 139ms	remaining: 511ms
214:	learn: 11.5586789	total: 140ms	remaining: 510ms
215:	learn: 11.5266294	total: 140ms	remaining: 509ms
216:	learn: 11.4983855	total: 141ms	remaining: 509ms
217:	learn: 11.4792076	total: 141ms	remaining: 507ms
218:	learn: 11.4537900	total: 142ms	remaining: 507ms
219:	learn: 11.4175184	total: 143ms	remaining: 506ms
220:	learn: 11.3855788	total: 143ms	remaining: 505ms
221:	learn: 11.3693530	total: 144ms	remaining: 504ms
222:	learn: 11.3504499	total: 144ms	remaining: 503ms
223:	learn: 11.3228405	total: 145ms	remaining: 502ms
224:	learn: 11.2954894	total: 146ms	remaining: 502ms
225:	learn: 11.2725195	total: 146ms	remaining:

441:	learn: 7.9490640	total: 275ms	remaining: 348ms
442:	learn: 7.9360843	total: 276ms	remaining: 347ms
443:	learn: 7.9210805	total: 277ms	remaining: 346ms
444:	learn: 7.9064842	total: 277ms	remaining: 346ms
445:	learn: 7.8962509	total: 278ms	remaining: 345ms
446:	learn: 7.8868618	total: 278ms	remaining: 344ms
447:	learn: 7.8695663	total: 280ms	remaining: 344ms
448:	learn: 7.8632055	total: 280ms	remaining: 344ms
449:	learn: 7.8547395	total: 280ms	remaining: 343ms
450:	learn: 7.8395993	total: 281ms	remaining: 342ms
451:	learn: 7.8303341	total: 282ms	remaining: 342ms
452:	learn: 7.8205733	total: 283ms	remaining: 341ms
453:	learn: 7.8067175	total: 283ms	remaining: 341ms
454:	learn: 7.7994757	total: 284ms	remaining: 340ms
455:	learn: 7.7914937	total: 284ms	remaining: 339ms
456:	learn: 7.7834234	total: 285ms	remaining: 338ms
457:	learn: 7.7746723	total: 285ms	remaining: 338ms
458:	learn: 7.7591725	total: 286ms	remaining: 337ms
459:	learn: 7.7497396	total: 286ms	remaining: 336ms
460:	learn: 

700:	learn: 5.9594121	total: 421ms	remaining: 179ms
701:	learn: 5.9544246	total: 421ms	remaining: 179ms
702:	learn: 5.9491010	total: 422ms	remaining: 178ms
703:	learn: 5.9434416	total: 423ms	remaining: 178ms
704:	learn: 5.9360945	total: 423ms	remaining: 177ms
705:	learn: 5.9322696	total: 424ms	remaining: 177ms
706:	learn: 5.9270733	total: 425ms	remaining: 176ms
707:	learn: 5.9190168	total: 425ms	remaining: 175ms
708:	learn: 5.9135767	total: 426ms	remaining: 175ms
709:	learn: 5.9072588	total: 427ms	remaining: 174ms
710:	learn: 5.9009111	total: 427ms	remaining: 174ms
711:	learn: 5.8960931	total: 428ms	remaining: 173ms
712:	learn: 5.8911168	total: 428ms	remaining: 172ms
713:	learn: 5.8856553	total: 429ms	remaining: 172ms
714:	learn: 5.8790670	total: 429ms	remaining: 171ms
715:	learn: 5.8755283	total: 430ms	remaining: 170ms
716:	learn: 5.8724728	total: 430ms	remaining: 170ms
717:	learn: 5.8672855	total: 431ms	remaining: 169ms
718:	learn: 5.8626085	total: 431ms	remaining: 169ms
719:	learn: 

943:	learn: 4.8764383	total: 562ms	remaining: 33.3ms
944:	learn: 4.8736223	total: 565ms	remaining: 32.9ms
945:	learn: 4.8704933	total: 566ms	remaining: 32.3ms
946:	learn: 4.8666064	total: 566ms	remaining: 31.7ms
947:	learn: 4.8637759	total: 567ms	remaining: 31.1ms
948:	learn: 4.8581494	total: 568ms	remaining: 30.5ms
949:	learn: 4.8555211	total: 569ms	remaining: 29.9ms
950:	learn: 4.8515237	total: 569ms	remaining: 29.3ms
951:	learn: 4.8482544	total: 570ms	remaining: 28.7ms
952:	learn: 4.8445220	total: 570ms	remaining: 28.1ms
953:	learn: 4.8422824	total: 571ms	remaining: 27.6ms
954:	learn: 4.8400887	total: 572ms	remaining: 27ms
955:	learn: 4.8372803	total: 573ms	remaining: 26.4ms
956:	learn: 4.8346574	total: 574ms	remaining: 25.8ms
957:	learn: 4.8325351	total: 574ms	remaining: 25.2ms
958:	learn: 4.8287122	total: 575ms	remaining: 24.6ms
959:	learn: 4.8259661	total: 576ms	remaining: 24ms
960:	learn: 4.8211606	total: 576ms	remaining: 23.4ms
961:	learn: 4.8182445	total: 577ms	remaining: 22.8

<ipython-input-44-cb3c29d8a275>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['kosdaq'] = kosdaq_pred
<ipython-input-44-cb3c29d8a275>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['kosdaq'] = test_kosdaq_pred


In [45]:
X_test.head()

,weekday,weeknum,kosdaq
215,0,44,997.251215
216,1,44,1007.362105
217,2,44,1005.700646
218,3,44,998.877414
219,4,44,995.523813


In [46]:
y_nasdaq = train_data['nasdaq']

# nasdaq 예측
cat.fit(X_train, y_nasdaq)
nasdaq_pred = cat.predict(X_train)
X_train['nasdaq'] = nasdaq_pred
test_nasdaq_pred = cat.predict(X_test)
X_test['nasdaq'] = test_nasdaq_pred
X_train.head()

Learning rate set to 0.032115
0:	learn: 702.2073904	total: 1.1ms	remaining: 1.09s
1:	learn: 687.1905119	total: 1.72ms	remaining: 860ms
2:	learn: 670.4840687	total: 2.89ms	remaining: 959ms
3:	learn: 655.9648667	total: 3.94ms	remaining: 980ms
4:	learn: 641.2278796	total: 14.1ms	remaining: 2.8s
5:	learn: 627.6265643	total: 15.4ms	remaining: 2.54s
6:	learn: 612.3196104	total: 16.1ms	remaining: 2.29s
7:	learn: 599.5012564	total: 17.3ms	remaining: 2.15s
8:	learn: 587.8975759	total: 18.4ms	remaining: 2.02s
9:	learn: 575.9014609	total: 19.4ms	remaining: 1.92s
10:	learn: 564.9480392	total: 20.5ms	remaining: 1.84s
11:	learn: 554.0210830	total: 21.3ms	remaining: 1.75s
12:	learn: 543.0691104	total: 22.3ms	remaining: 1.69s
13:	learn: 533.0953852	total: 23.3ms	remaining: 1.64s
14:	learn: 522.4369252	total: 24.3ms	remaining: 1.6s
15:	learn: 511.0556300	total: 25.5ms	remaining: 1.56s
16:	learn: 501.9651747	total: 26.8ms	remaining: 1.55s
17:	learn: 491.4308983	total: 27.9ms	remaining: 1.52s
18:	learn: 

279:	learn: 116.7184460	total: 304ms	remaining: 783ms
280:	learn: 116.4313848	total: 306ms	remaining: 782ms
281:	learn: 116.2872166	total: 307ms	remaining: 781ms
282:	learn: 116.2300454	total: 308ms	remaining: 780ms
283:	learn: 115.8579986	total: 309ms	remaining: 778ms
284:	learn: 115.3555518	total: 310ms	remaining: 777ms
285:	learn: 115.3121399	total: 312ms	remaining: 778ms
286:	learn: 115.0593650	total: 313ms	remaining: 778ms
287:	learn: 115.0033625	total: 314ms	remaining: 776ms
288:	learn: 114.6028782	total: 315ms	remaining: 775ms
289:	learn: 114.2853473	total: 316ms	remaining: 773ms
290:	learn: 114.0171817	total: 317ms	remaining: 772ms
291:	learn: 113.9616671	total: 318ms	remaining: 771ms
292:	learn: 113.8888561	total: 319ms	remaining: 770ms
293:	learn: 113.6989628	total: 320ms	remaining: 769ms
294:	learn: 113.6465728	total: 321ms	remaining: 767ms
295:	learn: 113.5941411	total: 322ms	remaining: 766ms
296:	learn: 113.5425335	total: 323ms	remaining: 765ms
297:	learn: 113.1787298	tota

580:	learn: 74.9101749	total: 626ms	remaining: 451ms
581:	learn: 74.8726250	total: 627ms	remaining: 450ms
582:	learn: 74.7989027	total: 628ms	remaining: 449ms
583:	learn: 74.6611150	total: 629ms	remaining: 448ms
584:	learn: 74.6090190	total: 630ms	remaining: 447ms
585:	learn: 74.4952706	total: 631ms	remaining: 446ms
586:	learn: 74.4772233	total: 632ms	remaining: 445ms
587:	learn: 74.3443433	total: 633ms	remaining: 444ms
588:	learn: 74.2362397	total: 634ms	remaining: 443ms
589:	learn: 74.0906953	total: 635ms	remaining: 441ms
590:	learn: 73.9755157	total: 636ms	remaining: 440ms
591:	learn: 73.9450716	total: 637ms	remaining: 439ms
592:	learn: 73.8329584	total: 638ms	remaining: 438ms
593:	learn: 73.7187197	total: 639ms	remaining: 437ms
594:	learn: 73.6114172	total: 640ms	remaining: 436ms
595:	learn: 73.5874953	total: 641ms	remaining: 435ms
596:	learn: 73.4900142	total: 642ms	remaining: 434ms
597:	learn: 73.2900322	total: 643ms	remaining: 433ms
598:	learn: 73.2144352	total: 645ms	remaining:

837:	learn: 56.1520092	total: 932ms	remaining: 180ms
838:	learn: 56.0837741	total: 935ms	remaining: 179ms
839:	learn: 56.0381229	total: 936ms	remaining: 178ms
840:	learn: 55.9690747	total: 937ms	remaining: 177ms
841:	learn: 55.9219936	total: 939ms	remaining: 176ms
842:	learn: 55.8165092	total: 940ms	remaining: 175ms
843:	learn: 55.7622695	total: 941ms	remaining: 174ms
844:	learn: 55.7442257	total: 942ms	remaining: 173ms
845:	learn: 55.6906050	total: 943ms	remaining: 172ms
846:	learn: 55.6210728	total: 945ms	remaining: 171ms
847:	learn: 55.5493122	total: 946ms	remaining: 170ms
848:	learn: 55.5098419	total: 947ms	remaining: 168ms
849:	learn: 55.4495174	total: 948ms	remaining: 167ms
850:	learn: 55.3861201	total: 949ms	remaining: 166ms
851:	learn: 55.2988770	total: 950ms	remaining: 165ms
852:	learn: 55.2692102	total: 952ms	remaining: 164ms
853:	learn: 55.2285724	total: 954ms	remaining: 163ms
854:	learn: 55.2055296	total: 955ms	remaining: 162ms
855:	learn: 55.1479923	total: 956ms	remaining:

<ipython-input-46-c1dff991700b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['nasdaq'] = nasdaq_pred
<ipython-input-46-c1dff991700b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['nasdaq'] = test_nasdaq_pred


,weekday,weeknum,kosdaq,nasdaq
0,0,1,978.915337,12739.544938
1,1,1,982.626223,12795.906815
2,2,1,983.790881,12775.417312
3,3,1,988.356503,13088.714644
4,4,1,985.220484,13166.907438


In [47]:
X_test.head()

,weekday,weeknum,kosdaq,nasdaq
215,0,44,997.251215,15235.094996
216,1,44,1007.362105,15225.543082
217,2,44,1005.700646,15270.071852
218,3,44,998.877414,15429.823511
219,4,44,995.523813,15439.403402


In [48]:
y_sp500 = train_data['sp500']

# sp500 예측
cat.fit(X_train, y_sp500)
sp500_pred = cat.predict(X_train)
X_train['sp500'] = sp500_pred
test_sp500_pred = cat.predict(X_test)
X_test['sp500'] = test_sp500_pred
X_train.head()

Learning rate set to 0.032115
0:	learn: 237.5081785	total: 4.77ms	remaining: 4.77s
1:	learn: 231.6408669	total: 6.97ms	remaining: 3.48s
2:	learn: 226.1051811	total: 8.12ms	remaining: 2.7s
3:	learn: 220.6399389	total: 9.4ms	remaining: 2.34s
4:	learn: 216.0343654	total: 10.5ms	remaining: 2.08s
5:	learn: 210.4838076	total: 13.2ms	remaining: 2.18s
6:	learn: 204.7728348	total: 14.1ms	remaining: 2s
7:	learn: 200.2513747	total: 17ms	remaining: 2.11s
8:	learn: 195.4598007	total: 18.9ms	remaining: 2.09s
9:	learn: 191.1825477	total: 20.4ms	remaining: 2.02s
10:	learn: 186.7127912	total: 21.5ms	remaining: 1.93s
11:	learn: 181.6926456	total: 22.7ms	remaining: 1.87s
12:	learn: 177.2522448	total: 23.9ms	remaining: 1.82s
13:	learn: 172.9475746	total: 25ms	remaining: 1.76s
14:	learn: 168.9520930	total: 26.9ms	remaining: 1.76s
15:	learn: 164.9989876	total: 28.5ms	remaining: 1.75s
16:	learn: 160.7983760	total: 30.5ms	remaining: 1.76s
17:	learn: 157.5134856	total: 32.7ms	remaining: 1.78s
18:	learn: 153.61

254:	learn: 17.2688318	total: 315ms	remaining: 921ms
255:	learn: 17.2386282	total: 317ms	remaining: 920ms
256:	learn: 17.2151826	total: 318ms	remaining: 918ms
257:	learn: 17.1736326	total: 319ms	remaining: 917ms
258:	learn: 17.1337136	total: 320ms	remaining: 915ms
259:	learn: 17.1121666	total: 321ms	remaining: 913ms
260:	learn: 17.0653754	total: 322ms	remaining: 911ms
261:	learn: 17.0063616	total: 323ms	remaining: 909ms
262:	learn: 16.9610654	total: 324ms	remaining: 907ms
263:	learn: 16.9268825	total: 325ms	remaining: 906ms
264:	learn: 16.8820175	total: 326ms	remaining: 905ms
265:	learn: 16.8376394	total: 327ms	remaining: 903ms
266:	learn: 16.7884096	total: 328ms	remaining: 901ms
267:	learn: 16.7733821	total: 329ms	remaining: 900ms
268:	learn: 16.7311232	total: 330ms	remaining: 898ms
269:	learn: 16.7165436	total: 332ms	remaining: 896ms
270:	learn: 16.6588117	total: 333ms	remaining: 895ms
271:	learn: 16.6404047	total: 334ms	remaining: 893ms
272:	learn: 16.6247753	total: 335ms	remaining:

504:	learn: 11.4615747	total: 630ms	remaining: 617ms
505:	learn: 11.4579917	total: 632ms	remaining: 617ms
506:	learn: 11.4479052	total: 633ms	remaining: 615ms
507:	learn: 11.4339565	total: 634ms	remaining: 614ms
508:	learn: 11.4306787	total: 635ms	remaining: 613ms
509:	learn: 11.4132212	total: 636ms	remaining: 611ms
510:	learn: 11.3974942	total: 638ms	remaining: 610ms
511:	learn: 11.3791873	total: 639ms	remaining: 609ms
512:	learn: 11.3756893	total: 640ms	remaining: 608ms
513:	learn: 11.3562981	total: 642ms	remaining: 607ms
514:	learn: 11.3546477	total: 643ms	remaining: 605ms
515:	learn: 11.3486874	total: 645ms	remaining: 605ms
516:	learn: 11.3470200	total: 646ms	remaining: 604ms
517:	learn: 11.3456030	total: 648ms	remaining: 603ms
518:	learn: 11.3291186	total: 653ms	remaining: 605ms
519:	learn: 11.3180486	total: 654ms	remaining: 604ms
520:	learn: 11.2902549	total: 656ms	remaining: 603ms
521:	learn: 11.2771810	total: 658ms	remaining: 602ms
522:	learn: 11.2566944	total: 661ms	remaining:

718:	learn: 8.7295902	total: 936ms	remaining: 366ms
719:	learn: 8.7159686	total: 938ms	remaining: 365ms
720:	learn: 8.7072805	total: 939ms	remaining: 363ms
721:	learn: 8.6956225	total: 941ms	remaining: 362ms
722:	learn: 8.6858304	total: 942ms	remaining: 361ms
723:	learn: 8.6848041	total: 943ms	remaining: 360ms
724:	learn: 8.6644626	total: 945ms	remaining: 358ms
725:	learn: 8.6434298	total: 946ms	remaining: 357ms
726:	learn: 8.6338115	total: 947ms	remaining: 356ms
727:	learn: 8.6139495	total: 949ms	remaining: 355ms
728:	learn: 8.6108606	total: 950ms	remaining: 353ms
729:	learn: 8.5988107	total: 952ms	remaining: 352ms
730:	learn: 8.5793438	total: 954ms	remaining: 351ms
731:	learn: 8.5727715	total: 955ms	remaining: 350ms
732:	learn: 8.5702423	total: 957ms	remaining: 348ms
733:	learn: 8.5508424	total: 960ms	remaining: 348ms
734:	learn: 8.5388141	total: 961ms	remaining: 346ms
735:	learn: 8.5268203	total: 962ms	remaining: 345ms
736:	learn: 8.5167021	total: 963ms	remaining: 344ms
737:	learn: 

932:	learn: 6.8215487	total: 1.24s	remaining: 89.3ms
933:	learn: 6.8163519	total: 1.25s	remaining: 88ms
934:	learn: 6.8122480	total: 1.25s	remaining: 86.7ms
935:	learn: 6.7965988	total: 1.25s	remaining: 85.3ms
936:	learn: 6.7910583	total: 1.25s	remaining: 84ms
937:	learn: 6.7871925	total: 1.25s	remaining: 82.6ms
938:	learn: 6.7851582	total: 1.25s	remaining: 81.3ms
939:	learn: 6.7703030	total: 1.25s	remaining: 80ms
940:	learn: 6.7602284	total: 1.25s	remaining: 78.7ms
941:	learn: 6.7519015	total: 1.26s	remaining: 77.3ms
942:	learn: 6.7496239	total: 1.26s	remaining: 76ms
943:	learn: 6.7421977	total: 1.26s	remaining: 74.7ms
944:	learn: 6.7388210	total: 1.26s	remaining: 73.4ms
945:	learn: 6.7297118	total: 1.26s	remaining: 72.1ms
946:	learn: 6.7267049	total: 1.26s	remaining: 70.7ms
947:	learn: 6.7247600	total: 1.26s	remaining: 69.4ms
948:	learn: 6.7210607	total: 1.27s	remaining: 68ms
949:	learn: 6.7132513	total: 1.27s	remaining: 66.7ms
950:	learn: 6.7094603	total: 1.27s	remaining: 65.4ms
951

<ipython-input-48-3ef180b30799>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['sp500'] = sp500_pred
<ipython-input-48-3ef180b30799>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['sp500'] = test_sp500_pred


,weekday,weeknum,kosdaq,nasdaq,sp500
0,0,1,978.915337,12739.544938,3705.665866
1,1,1,982.626223,12795.906815,3730.218202
2,2,1,983.790881,12775.417312,3743.356416
3,3,1,988.356503,13088.714644,3806.472623
4,4,1,985.220484,13166.907438,3820.928507


In [49]:
X_test.head()

,weekday,weeknum,kosdaq,nasdaq,sp500
215,0,44,997.251215,15235.094996,4561.993027
216,1,44,1007.362105,15225.543082,4568.170342
217,2,44,1005.700646,15270.071852,4560.706795
218,3,44,998.877414,15429.823511,4598.931569
219,4,44,995.523813,15439.403402,4601.296185


In [50]:
y_dow = train_data['dow']

# dow 예측
cat.fit(X_train, y_dow)
dow_pred = cat.predict(X_train)
X_train['dow'] = dow_pred
test_dow_pred = cat.predict(X_test)
X_test['dow'] = test_dow_pred
X_train.head()

Learning rate set to 0.032115
0:	learn: 1501.1690263	total: 1.56ms	remaining: 1.56s
1:	learn: 1465.4104899	total: 4.23ms	remaining: 2.11s
2:	learn: 1432.5165597	total: 6.07ms	remaining: 2.02s
3:	learn: 1399.6925685	total: 7.5ms	remaining: 1.87s
4:	learn: 1370.1578450	total: 8.99ms	remaining: 1.79s
5:	learn: 1334.2383712	total: 10.2ms	remaining: 1.69s
6:	learn: 1306.3325803	total: 11.4ms	remaining: 1.62s
7:	learn: 1277.7616847	total: 13.8ms	remaining: 1.71s
8:	learn: 1246.8212442	total: 15ms	remaining: 1.65s
9:	learn: 1213.9189501	total: 16.3ms	remaining: 1.61s
10:	learn: 1182.2436675	total: 17.6ms	remaining: 1.58s
11:	learn: 1156.1830623	total: 18.6ms	remaining: 1.53s
12:	learn: 1132.1982189	total: 19.4ms	remaining: 1.48s
13:	learn: 1104.2523634	total: 20.4ms	remaining: 1.44s
14:	learn: 1080.4711053	total: 21.4ms	remaining: 1.41s
15:	learn: 1054.2527318	total: 22.1ms	remaining: 1.36s
16:	learn: 1027.0054276	total: 23.2ms	remaining: 1.34s
17:	learn: 1005.0970490	total: 24.3ms	remaining:

211:	learn: 157.5043961	total: 319ms	remaining: 1.19s
212:	learn: 157.0774446	total: 321ms	remaining: 1.19s
213:	learn: 156.6044637	total: 323ms	remaining: 1.19s
214:	learn: 156.4944780	total: 324ms	remaining: 1.18s
215:	learn: 156.1490543	total: 325ms	remaining: 1.18s
216:	learn: 155.7001796	total: 327ms	remaining: 1.18s
217:	learn: 155.2572493	total: 329ms	remaining: 1.18s
218:	learn: 154.5628675	total: 330ms	remaining: 1.18s
219:	learn: 154.3505993	total: 331ms	remaining: 1.17s
220:	learn: 153.6691267	total: 333ms	remaining: 1.17s
221:	learn: 152.9828434	total: 335ms	remaining: 1.17s
222:	learn: 152.5253072	total: 336ms	remaining: 1.17s
223:	learn: 151.9443609	total: 338ms	remaining: 1.17s
224:	learn: 151.8526125	total: 339ms	remaining: 1.17s
225:	learn: 151.3812677	total: 340ms	remaining: 1.16s
226:	learn: 150.8861423	total: 341ms	remaining: 1.16s
227:	learn: 150.7961281	total: 343ms	remaining: 1.16s
228:	learn: 150.4883543	total: 344ms	remaining: 1.16s
229:	learn: 149.9557287	tota

425:	learn: 94.4024232	total: 642ms	remaining: 866ms
426:	learn: 94.2112196	total: 645ms	remaining: 866ms
427:	learn: 93.9391991	total: 647ms	remaining: 865ms
428:	learn: 93.8651479	total: 649ms	remaining: 864ms
429:	learn: 93.7411834	total: 650ms	remaining: 862ms
430:	learn: 93.6089151	total: 652ms	remaining: 861ms
431:	learn: 93.5057534	total: 654ms	remaining: 860ms
432:	learn: 93.3744532	total: 657ms	remaining: 860ms
433:	learn: 93.3173170	total: 660ms	remaining: 861ms
434:	learn: 93.1588759	total: 662ms	remaining: 860ms
435:	learn: 93.0812923	total: 663ms	remaining: 858ms
436:	learn: 93.0002870	total: 665ms	remaining: 856ms
437:	learn: 92.8824935	total: 666ms	remaining: 855ms
438:	learn: 92.8125132	total: 667ms	remaining: 853ms
439:	learn: 92.5419089	total: 669ms	remaining: 851ms
440:	learn: 92.4187129	total: 670ms	remaining: 850ms
441:	learn: 92.2527946	total: 673ms	remaining: 849ms
442:	learn: 91.8990014	total: 674ms	remaining: 847ms
443:	learn: 91.7762094	total: 676ms	remaining:

606:	learn: 70.1987749	total: 957ms	remaining: 620ms
607:	learn: 69.9241458	total: 959ms	remaining: 619ms
608:	learn: 69.7858101	total: 961ms	remaining: 617ms
609:	learn: 69.6783500	total: 963ms	remaining: 616ms
610:	learn: 69.5591175	total: 968ms	remaining: 616ms
611:	learn: 69.5325436	total: 970ms	remaining: 615ms
612:	learn: 69.4182629	total: 973ms	remaining: 614ms
613:	learn: 69.3172531	total: 975ms	remaining: 613ms
614:	learn: 69.2040184	total: 978ms	remaining: 612ms
615:	learn: 69.0692143	total: 983ms	remaining: 613ms
616:	learn: 68.9360284	total: 985ms	remaining: 611ms
617:	learn: 68.8959801	total: 986ms	remaining: 610ms
618:	learn: 68.7180481	total: 988ms	remaining: 608ms
619:	learn: 68.6860200	total: 990ms	remaining: 607ms
620:	learn: 68.6380851	total: 992ms	remaining: 606ms
621:	learn: 68.5349997	total: 994ms	remaining: 604ms
622:	learn: 68.4056479	total: 997ms	remaining: 603ms
623:	learn: 68.3136631	total: 998ms	remaining: 601ms
624:	learn: 68.2885099	total: 999ms	remaining:

785:	learn: 53.3359603	total: 1.27s	remaining: 347ms
786:	learn: 53.2764911	total: 1.27s	remaining: 345ms
787:	learn: 53.1830239	total: 1.28s	remaining: 344ms
788:	learn: 53.0410648	total: 1.28s	remaining: 342ms
789:	learn: 52.9504010	total: 1.28s	remaining: 340ms
790:	learn: 52.8942669	total: 1.28s	remaining: 339ms
791:	learn: 52.8309221	total: 1.28s	remaining: 337ms
792:	learn: 52.6934474	total: 1.29s	remaining: 336ms
793:	learn: 52.6414796	total: 1.29s	remaining: 334ms
794:	learn: 52.6038402	total: 1.29s	remaining: 333ms
795:	learn: 52.5112859	total: 1.29s	remaining: 331ms
796:	learn: 52.4588539	total: 1.29s	remaining: 330ms
797:	learn: 52.3783714	total: 1.3s	remaining: 328ms
798:	learn: 52.3131819	total: 1.3s	remaining: 327ms
799:	learn: 52.2643258	total: 1.3s	remaining: 325ms
800:	learn: 52.1868902	total: 1.3s	remaining: 324ms
801:	learn: 52.1135464	total: 1.3s	remaining: 322ms
802:	learn: 51.9835103	total: 1.3s	remaining: 320ms
803:	learn: 51.9453265	total: 1.31s	remaining: 319ms

949:	learn: 43.3315059	total: 1.59s	remaining: 83.6ms
950:	learn: 43.2870522	total: 1.59s	remaining: 82ms
951:	learn: 43.2399947	total: 1.59s	remaining: 80.3ms
952:	learn: 43.1520704	total: 1.59s	remaining: 78.6ms
953:	learn: 43.0999194	total: 1.6s	remaining: 77ms
954:	learn: 43.0570270	total: 1.6s	remaining: 75.3ms
955:	learn: 43.0289751	total: 1.6s	remaining: 73.7ms
956:	learn: 43.0058217	total: 1.6s	remaining: 72ms
957:	learn: 42.8955054	total: 1.6s	remaining: 70.4ms
958:	learn: 42.8106561	total: 1.61s	remaining: 68.7ms
959:	learn: 42.7946941	total: 1.61s	remaining: 67.1ms
960:	learn: 42.7491911	total: 1.61s	remaining: 65.4ms
961:	learn: 42.7042888	total: 1.61s	remaining: 63.7ms
962:	learn: 42.6605125	total: 1.61s	remaining: 62.1ms
963:	learn: 42.5800781	total: 1.62s	remaining: 60.4ms
964:	learn: 42.5000234	total: 1.62s	remaining: 58.7ms
965:	learn: 42.4837774	total: 1.62s	remaining: 57.1ms
966:	learn: 42.4080008	total: 1.62s	remaining: 55.4ms
967:	learn: 42.3451830	total: 1.62s	rem

<ipython-input-50-12d6b008ada0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['dow'] = dow_pred
<ipython-input-50-12d6b008ada0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['dow'] = test_dow_pred


,weekday,weeknum,kosdaq,nasdaq,sp500,dow
0,0,1,978.915337,12739.544938,3705.665866,30236.332234
1,1,1,982.626223,12795.906815,3730.218202,30406.949445
2,2,1,983.790881,12775.417312,3743.356416,30797.629907
3,3,1,988.356503,13088.714644,3806.472623,31072.783803
4,4,1,985.220484,13166.907438,3820.928507,31095.401032


In [51]:
X_test.head()

,weekday,weeknum,kosdaq,nasdaq,sp500,dow
215,0,44,997.251215,15235.094996,4561.993027,35733.275093
216,1,44,1007.362105,15225.543082,4568.170342,35719.783313
217,2,44,1005.700646,15270.071852,4560.706795,35560.624312
218,3,44,998.877414,15429.823511,4598.931569,35753.857879
219,4,44,995.523813,15439.403402,4601.296185,35783.246717


In [52]:
y_kospi = train_data['kospi']

# kospi 예측
cat.fit(X_train, y_kospi)
kospi_pred = cat.predict(X_train)
X_train['kospi'] = kospi_pred
test_kospi_pred = cat.predict(X_test)
X_test['kospi'] = test_kospi_pred
X_train.head()

Learning rate set to 0.032115
0:	learn: 92.4728492	total: 1.52ms	remaining: 1.52s
1:	learn: 90.6415215	total: 2.86ms	remaining: 1.43s
2:	learn: 88.8471918	total: 4.14ms	remaining: 1.38s
3:	learn: 87.0065445	total: 5.71ms	remaining: 1.42s
4:	learn: 85.2240031	total: 6.94ms	remaining: 1.38s
5:	learn: 83.4419944	total: 8.17ms	remaining: 1.35s
6:	learn: 81.9396858	total: 9.83ms	remaining: 1.39s
7:	learn: 80.2838525	total: 10.9ms	remaining: 1.35s
8:	learn: 78.9041005	total: 12.1ms	remaining: 1.34s
9:	learn: 77.2926235	total: 13.2ms	remaining: 1.3s
10:	learn: 75.7765130	total: 14.2ms	remaining: 1.28s
11:	learn: 74.4661704	total: 15.4ms	remaining: 1.27s
12:	learn: 72.9387431	total: 17ms	remaining: 1.29s
13:	learn: 71.3835990	total: 18.4ms	remaining: 1.3s
14:	learn: 70.0366916	total: 19.7ms	remaining: 1.29s
15:	learn: 68.6682587	total: 20.7ms	remaining: 1.27s
16:	learn: 67.4734691	total: 21.8ms	remaining: 1.26s
17:	learn: 66.3019883	total: 22.8ms	remaining: 1.25s
18:	learn: 65.1738296	total: 2

206:	learn: 18.2481895	total: 312ms	remaining: 1.2s
207:	learn: 18.1911237	total: 315ms	remaining: 1.2s
208:	learn: 18.1334190	total: 317ms	remaining: 1.2s
209:	learn: 18.0892887	total: 318ms	remaining: 1.2s
210:	learn: 18.0252319	total: 321ms	remaining: 1.2s
211:	learn: 17.9928522	total: 323ms	remaining: 1.2s
212:	learn: 17.9208850	total: 324ms	remaining: 1.2s
213:	learn: 17.8344882	total: 326ms	remaining: 1.2s
214:	learn: 17.7995363	total: 328ms	remaining: 1.2s
215:	learn: 17.7414994	total: 331ms	remaining: 1.2s
216:	learn: 17.6747879	total: 333ms	remaining: 1.2s
217:	learn: 17.6256775	total: 336ms	remaining: 1.2s
218:	learn: 17.5756322	total: 338ms	remaining: 1.2s
219:	learn: 17.5305551	total: 340ms	remaining: 1.2s
220:	learn: 17.4926398	total: 341ms	remaining: 1.2s
221:	learn: 17.4624303	total: 343ms	remaining: 1.2s
222:	learn: 17.3953521	total: 345ms	remaining: 1.2s
223:	learn: 17.3601217	total: 347ms	remaining: 1.2s
224:	learn: 17.3288143	total: 348ms	remaining: 1.2s
225:	learn: 

369:	learn: 13.3573805	total: 637ms	remaining: 1.08s
370:	learn: 13.3391235	total: 639ms	remaining: 1.08s
371:	learn: 13.3284960	total: 641ms	remaining: 1.08s
372:	learn: 13.3075526	total: 643ms	remaining: 1.08s
373:	learn: 13.2717635	total: 648ms	remaining: 1.08s
374:	learn: 13.2527795	total: 653ms	remaining: 1.09s
375:	learn: 13.2483535	total: 655ms	remaining: 1.09s
376:	learn: 13.2430667	total: 656ms	remaining: 1.08s
377:	learn: 13.2264273	total: 658ms	remaining: 1.08s
378:	learn: 13.2192416	total: 661ms	remaining: 1.08s
379:	learn: 13.2006885	total: 663ms	remaining: 1.08s
380:	learn: 13.1766804	total: 665ms	remaining: 1.08s
381:	learn: 13.1332259	total: 669ms	remaining: 1.08s
382:	learn: 13.1228628	total: 670ms	remaining: 1.08s
383:	learn: 13.1188226	total: 672ms	remaining: 1.08s
384:	learn: 13.1147377	total: 674ms	remaining: 1.08s
385:	learn: 13.0812650	total: 676ms	remaining: 1.07s
386:	learn: 13.0669660	total: 678ms	remaining: 1.07s
387:	learn: 13.0415516	total: 682ms	remaining:

533:	learn: 10.0578972	total: 963ms	remaining: 840ms
534:	learn: 10.0369164	total: 966ms	remaining: 840ms
535:	learn: 10.0279763	total: 970ms	remaining: 839ms
536:	learn: 10.0087668	total: 973ms	remaining: 839ms
537:	learn: 10.0034576	total: 975ms	remaining: 837ms
538:	learn: 9.9770318	total: 977ms	remaining: 836ms
539:	learn: 9.9717315	total: 980ms	remaining: 835ms
540:	learn: 9.9633817	total: 982ms	remaining: 833ms
541:	learn: 9.9423983	total: 983ms	remaining: 831ms
542:	learn: 9.9195104	total: 986ms	remaining: 830ms
543:	learn: 9.8936779	total: 988ms	remaining: 828ms
544:	learn: 9.8808381	total: 989ms	remaining: 826ms
545:	learn: 9.8758190	total: 991ms	remaining: 824ms
546:	learn: 9.8456275	total: 993ms	remaining: 822ms
547:	learn: 9.8321025	total: 994ms	remaining: 820ms
548:	learn: 9.8123349	total: 996ms	remaining: 818ms
549:	learn: 9.8009129	total: 998ms	remaining: 817ms
550:	learn: 9.7805303	total: 1s	remaining: 815ms
551:	learn: 9.7746233	total: 1s	remaining: 813ms
552:	learn: 9

697:	learn: 7.7412545	total: 1.29s	remaining: 557ms
698:	learn: 7.7273719	total: 1.29s	remaining: 556ms
699:	learn: 7.7217873	total: 1.29s	remaining: 555ms
700:	learn: 7.7027276	total: 1.3s	remaining: 553ms
701:	learn: 7.6841974	total: 1.3s	remaining: 551ms
702:	learn: 7.6742563	total: 1.3s	remaining: 550ms
703:	learn: 7.6701362	total: 1.3s	remaining: 549ms
704:	learn: 7.6580499	total: 1.31s	remaining: 547ms
705:	learn: 7.6525491	total: 1.31s	remaining: 545ms
706:	learn: 7.6338383	total: 1.31s	remaining: 543ms
707:	learn: 7.6255037	total: 1.31s	remaining: 542ms
708:	learn: 7.6220707	total: 1.31s	remaining: 540ms
709:	learn: 7.6086807	total: 1.32s	remaining: 539ms
710:	learn: 7.5911611	total: 1.32s	remaining: 537ms
711:	learn: 7.5794468	total: 1.32s	remaining: 536ms
712:	learn: 7.5657710	total: 1.33s	remaining: 534ms
713:	learn: 7.5577573	total: 1.33s	remaining: 533ms
714:	learn: 7.5472998	total: 1.33s	remaining: 532ms
715:	learn: 7.5393831	total: 1.34s	remaining: 531ms
716:	learn: 7.52

898:	learn: 5.7521986	total: 1.61s	remaining: 181ms
899:	learn: 5.7416016	total: 1.61s	remaining: 179ms
900:	learn: 5.7272929	total: 1.61s	remaining: 177ms
901:	learn: 5.7191901	total: 1.61s	remaining: 175ms
902:	learn: 5.7088012	total: 1.61s	remaining: 174ms
903:	learn: 5.7027932	total: 1.62s	remaining: 172ms
904:	learn: 5.6854690	total: 1.62s	remaining: 170ms
905:	learn: 5.6808299	total: 1.62s	remaining: 168ms
906:	learn: 5.6797456	total: 1.62s	remaining: 166ms
907:	learn: 5.6673636	total: 1.62s	remaining: 165ms
908:	learn: 5.6552075	total: 1.63s	remaining: 163ms
909:	learn: 5.6442864	total: 1.63s	remaining: 161ms
910:	learn: 5.6302345	total: 1.63s	remaining: 159ms
911:	learn: 5.6205462	total: 1.63s	remaining: 157ms
912:	learn: 5.6116654	total: 1.63s	remaining: 155ms
913:	learn: 5.6003971	total: 1.63s	remaining: 154ms
914:	learn: 5.5907073	total: 1.63s	remaining: 152ms
915:	learn: 5.5769045	total: 1.63s	remaining: 150ms
916:	learn: 5.5695947	total: 1.64s	remaining: 148ms
917:	learn: 

<ipython-input-52-815d70540fc9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['kospi'] = kospi_pred
<ipython-input-52-815d70540fc9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['kospi'] = test_kospi_pred


,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,0,1,978.915337,12739.544938,3705.665866,30236.332234,2947.030536
1,1,1,982.626223,12795.906815,3730.218202,30406.949445,2987.528251
2,2,1,983.790881,12775.417312,3743.356416,30797.629907,2971.424894
3,3,1,988.356503,13088.714644,3806.472623,31072.783803,3040.073246
4,4,1,985.220484,13166.907438,3820.928507,31095.401032,3149.161503


In [53]:
X_test.head()

,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
215,0,44,997.251215,15235.094996,4561.993027,35733.275093,3017.783822
216,1,44,1007.362105,15225.543082,4568.170342,35719.783313,3048.664501
217,2,44,1005.700646,15270.071852,4560.706795,35560.624312,3028.905584
218,3,44,998.877414,15429.823511,4598.931569,35753.857879,3007.527283
219,4,44,995.523813,15439.403402,4601.296185,35783.246717,2975.408320


In [54]:
# 종가 예측
cat.fit(X_train, y_train)
close_pred = cat.predict(X_train)
X_train['Close'] = close_pred
test_close_pred = cat.predict(X_test)
X_test['Close'] = test_close_pred
X_train.head()

Learning rate set to 0.032115
0:	learn: 4411.8015511	total: 2.85ms	remaining: 2.84s
1:	learn: 4313.7936006	total: 6.18ms	remaining: 3.08s
2:	learn: 4216.4292972	total: 10.1ms	remaining: 3.37s
3:	learn: 4120.3695794	total: 13.4ms	remaining: 3.34s
4:	learn: 4045.0190546	total: 15.7ms	remaining: 3.13s
5:	learn: 3967.8555973	total: 17.5ms	remaining: 2.89s
6:	learn: 3885.9223252	total: 19.5ms	remaining: 2.77s
7:	learn: 3809.2942013	total: 21.1ms	remaining: 2.62s
8:	learn: 3736.6555943	total: 22.8ms	remaining: 2.51s
9:	learn: 3664.4809584	total: 26.6ms	remaining: 2.63s
10:	learn: 3587.2558811	total: 29.5ms	remaining: 2.65s
11:	learn: 3515.9588938	total: 31.5ms	remaining: 2.59s
12:	learn: 3445.8837609	total: 33.2ms	remaining: 2.52s
13:	learn: 3371.3504652	total: 35.2ms	remaining: 2.48s
14:	learn: 3288.9416393	total: 36.9ms	remaining: 2.42s
15:	learn: 3221.0094841	total: 38.7ms	remaining: 2.38s
16:	learn: 3165.6905606	total: 40.9ms	remaining: 2.37s
17:	learn: 3107.7780729	total: 42.1ms	remaini

162:	learn: 748.5967881	total: 329ms	remaining: 1.69s
163:	learn: 745.5199062	total: 332ms	remaining: 1.69s
164:	learn: 742.9427767	total: 334ms	remaining: 1.69s
165:	learn: 740.8056549	total: 336ms	remaining: 1.69s
166:	learn: 738.2010327	total: 338ms	remaining: 1.69s
167:	learn: 734.8380279	total: 340ms	remaining: 1.68s
168:	learn: 732.9028004	total: 342ms	remaining: 1.68s
169:	learn: 729.9916958	total: 343ms	remaining: 1.68s
170:	learn: 726.9242301	total: 345ms	remaining: 1.67s
171:	learn: 724.1935568	total: 348ms	remaining: 1.67s
172:	learn: 721.6842709	total: 349ms	remaining: 1.67s
173:	learn: 720.1704432	total: 350ms	remaining: 1.66s
174:	learn: 717.7025834	total: 352ms	remaining: 1.66s
175:	learn: 715.0886352	total: 354ms	remaining: 1.66s
176:	learn: 710.4037163	total: 355ms	remaining: 1.65s
177:	learn: 707.9183909	total: 357ms	remaining: 1.65s
178:	learn: 705.8084080	total: 358ms	remaining: 1.64s
179:	learn: 702.6033114	total: 360ms	remaining: 1.64s
180:	learn: 700.8939902	tota

334:	learn: 436.6782719	total: 665ms	remaining: 1.32s
335:	learn: 435.5918899	total: 667ms	remaining: 1.32s
336:	learn: 434.2802775	total: 670ms	remaining: 1.32s
337:	learn: 433.2336455	total: 672ms	remaining: 1.32s
338:	learn: 432.5262286	total: 675ms	remaining: 1.32s
339:	learn: 431.3617141	total: 678ms	remaining: 1.31s
340:	learn: 430.7248429	total: 680ms	remaining: 1.31s
341:	learn: 430.0607459	total: 683ms	remaining: 1.31s
342:	learn: 428.2394325	total: 686ms	remaining: 1.31s
343:	learn: 427.2002954	total: 690ms	remaining: 1.31s
344:	learn: 425.6528010	total: 693ms	remaining: 1.32s
345:	learn: 424.1975126	total: 696ms	remaining: 1.31s
346:	learn: 423.7413886	total: 698ms	remaining: 1.31s
347:	learn: 423.0270278	total: 702ms	remaining: 1.31s
348:	learn: 422.4324687	total: 704ms	remaining: 1.31s
349:	learn: 421.6627229	total: 710ms	remaining: 1.32s
350:	learn: 420.6226435	total: 712ms	remaining: 1.31s
351:	learn: 419.7537318	total: 714ms	remaining: 1.31s
352:	learn: 419.4905807	tota

522:	learn: 289.2348949	total: 1.16s	remaining: 1.05s
523:	learn: 288.1380310	total: 1.16s	remaining: 1.05s
524:	learn: 287.7848037	total: 1.16s	remaining: 1.05s
525:	learn: 287.0621809	total: 1.17s	remaining: 1.05s
526:	learn: 286.4976790	total: 1.17s	remaining: 1.05s
527:	learn: 285.5798449	total: 1.18s	remaining: 1.05s
528:	learn: 284.7581198	total: 1.18s	remaining: 1.05s
529:	learn: 284.6844163	total: 1.19s	remaining: 1.05s
530:	learn: 283.7775894	total: 1.19s	remaining: 1.05s
531:	learn: 283.5498227	total: 1.19s	remaining: 1.05s
532:	learn: 282.9383621	total: 1.19s	remaining: 1.04s
533:	learn: 282.0774078	total: 1.2s	remaining: 1.04s
534:	learn: 281.6861691	total: 1.2s	remaining: 1.04s
535:	learn: 281.0580271	total: 1.2s	remaining: 1.04s
536:	learn: 280.6860653	total: 1.2s	remaining: 1.04s
537:	learn: 280.6010920	total: 1.21s	remaining: 1.03s
538:	learn: 280.2414571	total: 1.21s	remaining: 1.03s
539:	learn: 279.4567107	total: 1.21s	remaining: 1.03s
540:	learn: 278.4050105	total: 1

738:	learn: 200.1316417	total: 1.65s	remaining: 583ms
739:	learn: 199.5830620	total: 1.65s	remaining: 581ms
740:	learn: 198.8417138	total: 1.65s	remaining: 578ms
741:	learn: 198.4750272	total: 1.66s	remaining: 576ms
742:	learn: 198.1194819	total: 1.66s	remaining: 574ms
743:	learn: 197.5473387	total: 1.66s	remaining: 571ms
744:	learn: 197.2326695	total: 1.66s	remaining: 569ms
745:	learn: 196.7043366	total: 1.67s	remaining: 567ms
746:	learn: 196.3599712	total: 1.67s	remaining: 565ms
747:	learn: 196.0375304	total: 1.67s	remaining: 563ms
748:	learn: 195.7743870	total: 1.67s	remaining: 561ms
749:	learn: 195.2627131	total: 1.68s	remaining: 559ms
750:	learn: 194.6591054	total: 1.68s	remaining: 557ms
751:	learn: 194.3342078	total: 1.68s	remaining: 555ms
752:	learn: 193.6052845	total: 1.69s	remaining: 553ms
753:	learn: 193.1889683	total: 1.69s	remaining: 550ms
754:	learn: 192.8032185	total: 1.69s	remaining: 548ms
755:	learn: 192.4637801	total: 1.69s	remaining: 546ms
756:	learn: 192.0768807	tota

975:	learn: 136.8268333	total: 2.14s	remaining: 52.6ms
976:	learn: 136.8013771	total: 2.14s	remaining: 50.4ms
977:	learn: 136.6183845	total: 2.14s	remaining: 48.2ms
978:	learn: 136.4314144	total: 2.15s	remaining: 46ms
979:	learn: 136.3146664	total: 2.15s	remaining: 43.8ms
980:	learn: 136.1984829	total: 2.15s	remaining: 41.6ms
981:	learn: 136.0546016	total: 2.15s	remaining: 39.4ms
982:	learn: 135.7264175	total: 2.15s	remaining: 37.2ms
983:	learn: 135.4391394	total: 2.15s	remaining: 35ms
984:	learn: 135.2027764	total: 2.16s	remaining: 32.8ms
985:	learn: 135.0159497	total: 2.16s	remaining: 30.6ms
986:	learn: 134.6396797	total: 2.16s	remaining: 28.5ms
987:	learn: 134.4436387	total: 2.16s	remaining: 26.3ms
988:	learn: 134.1452233	total: 2.16s	remaining: 24.1ms
989:	learn: 134.0261540	total: 2.17s	remaining: 21.9ms
990:	learn: 133.7577285	total: 2.17s	remaining: 19.7ms
991:	learn: 133.5458737	total: 2.17s	remaining: 17.5ms
992:	learn: 133.2113385	total: 2.17s	remaining: 15.3ms
993:	learn: 13

<ipython-input-54-0f2bd045dab0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Close'] = close_pred
<ipython-input-54-0f2bd045dab0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Close'] = test_close_pred


,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,Close
0,0,1,978.915337,12739.544938,3705.665866,30236.332234,2947.030536,83047.501276
1,1,1,982.626223,12795.906815,3730.218202,30406.949445,2987.528251,83781.263934
2,2,1,983.790881,12775.417312,3743.356416,30797.629907,2971.424894,82299.606541
3,3,1,988.356503,13088.714644,3806.472623,31072.783803,3040.073246,82977.415095
4,4,1,985.220484,13166.907438,3820.928507,31095.401032,3149.161503,88711.267667


In [55]:
X_test.head()

,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,Close
215,0,44,997.251215,15235.094996,4561.993027,35733.275093,3017.783822,70254.347296
216,1,44,1007.362105,15225.543082,4568.170342,35719.783313,3048.664501,71078.743023
217,2,44,1005.700646,15270.071852,4560.706795,35560.624312,3028.905584,70227.757467
218,3,44,998.877414,15429.823511,4598.931569,35753.857879,3007.527283,70497.314323
219,4,44,995.523813,15439.403402,4601.296185,35783.246717,2975.408320,69905.846127


In [56]:
def evaluation_index(pred, y):
    y_mean = y.mean()
    
    # MSE
    mse = ( (y - pred) ** 2 ).sum() / len(pred)
        
    # RMSE
    rmse = mse ** (1/2)
    
    # MAE
    mae = ( abs( y - pred ) ).sum() / len(pred)
    
    # R2
    y_mean = y.mean()
    upper = ( (y - pred) ** 2 ).sum()
    lower = ( (y - y_mean) ** 2 ).sum()
    fraction = upper / lower
    r2 = 1 - fraction
    
    #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
    nmae = ( abs( y - pred ) / y ).sum() / len(pred)
    
    # 평가지표 출력
    print(f"R2 = {r2}")
    print(f"MSE = {mse}")
    print(f"RMSE = {rmse}")
    print(f"MAE = {mae}")
    print(f"NMAE = {nmae}")
    print(f"NMAE * 100 = {nmae*100}")

In [57]:
evaluation_index(X_train['Close'], y_train)

R2 = 0.999152132073817
MSE = 17240.681582786456
RMSE = 131.30377596545523
MAE = 109.93153584968971
NMAE = 0.0013761520673230582
NMAE * 100 = 0.13761520673230582


In [58]:
evaluation_index(X_test['Close'], y_test)

R2 = 0.707249464275091
MSE = 85951.55728883327
RMSE = 293.1749602009579
MAE = 268.9372712180222
NMAE = 0.0038104752941640853
NMAE * 100 = 0.3810475294164085
